# Preanálisis

In [1]:
# Librerías de siempre
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import datetime
from time import time
import statsmodels.api as sm
import itertools
from random import shuffle
from statsmodels.formula.api import ols
from scipy import stats

In [2]:
def leer_efectos(datadir,
                 ligas):
    dict_replaces = {'_mean': '',
                     '_':' ',
                     'dificil':'Difícil',
                     'dificl' : 'Difícil',
                     'facil': 'Fácil',
                     'regular' : 'Regular',
                     'prom': 'Promedio',
                     'posicion': 'Posición',
                     'primer' : 'Primer',
                     'ronda' : 'Ronda',
                     'gral': 'Gral.',
                     'categoria': 'Categoría',
                     'local': 'Local',
                     'visita': 'Visita',
                     'general' : 'Gral.',
                     'corr' : 'Correg.',
                     'perc' : '%',
                     ' p': ' Primeros ',
                     ' c': ' Corte ',
                     'lv' : 'LV',
                     'elo' : 'ELO',
                     'Primer Fácil C': 'Primer Fácil LV C',
                     'Primer Difícil C': 'Primer Difícil LV C'}
    count = 0
    for j in range(len(ligas)):
        print("Leyendo: %s                             " % ligas[j], end = '\t\r')
        df_efecto = pd.read_excel(os.path.join(datadir, ligas[j] + '.xlsx'),
                                  sheet_name = ligas[j])
        cols_categoria = [i for i in df_efecto.columns if 'categoria' in i]
        for col in cols_categoria:
            df_efecto[col] = df_efecto[col].astype('category')
            df_efecto[col].cat.reorder_categories(['regular', 'facil','dificil'], inplace = True)
        if count < 1:
            lista_columnas = [i for i in df_efecto.columns.tolist() if i not in ['equipo','position','Torneo','Pais']
                             and '.' not in i]
#             lista_columnas = [i for i in df_efecto.columns.tolist() if 'perc' in i 
#                               and '.' not in i]
            variables = lista_columnas
            for key, value in dict_replaces.items():
                variables = [i.replace(key,value) for i in variables]
            dict_columns = dict(zip(variables, lista_columnas))
            dfs_efecto = []
            count = count + 1
        dfs_efecto.append(df_efecto)
    return dfs_efecto, variables, dict_columns

In [3]:
def significancia_variables(mod,var,replace_spaces = True, std_err = True):
    coef = "%.3f" % (np.round(mod.params[var], decimals = 3))
    se = "%.3f" % (np.round(mod.bse[var], decimals = 3))
    if mod.params[var] > 0:
        output = '~' + str(coef)                 
    else:
        output = str(coef)
    if std_err:
        output = output + ' (' + str(se) + ')'   
    pval = np.round(mod.pvalues[var], 3)
    if pval < 0.01:
        output = output + '***'
    elif pval < 0.05 and pval >= 0.01:
        output = output + '**~'
    elif pval < 0.1 and pval >=0.05:
        output = output + '*~~'
    else:
        output = output + '~~~'
    if replace_spaces:
        output = output.replace("~"," ")
    return output

## ANOVA de 1 vía para posiciones

La idea de esto es ver si hay diferencias en términos de posición entre los equipos que son asignados como fáciles o difíciles

In [4]:
def anova_1via_categorias(datadir,ligas, outputdir, outputfile, column_condition):
    dfs_efecto, variables, dict_columns = leer_efectos(datadir = datadir,
                                                       ligas = ligas)
    reverse_dict = inv_map = {v: k for k, v in dict_columns.items()}
    ys = [i for i in dfs_efecto[0].columns.tolist() if 'position' in i or 'points' in i]
    dfs_resumen = []
    for y in ys:
#         print(y)
        formula = '%s ~ C(equipo)' % y
        data = []
        for i in range(len(ligas)):
            resultados = []
            liga = ligas[i]
            df_efecto = dfs_efecto[i]
            if i < 1:
                cols_categorias = [j for j in df_efecto.columns if column_condition(j)]
                columna_var = []
                for col in cols_categorias:
                    for cat in ['facil','dificil']:
                        columna_var.append('%s %s' % (reverse_dict[col], cat))  
                data.append(columna_var)
            for col in cols_categorias:
                for cat in ['facil','dificil']:
                    model = ols(formula, data = df_efecto[df_efecto[col] == cat]).fit()
                    aov_table = sm.stats.anova_lm(model, typ=2)
                    resultados.append(np.round(aov_table['PR(>F)'][0], decimals = 3))
            data.append(resultados)
        colnames = ['Categoria'] + ligas
        data_dict = dict(zip(colnames, data))
        df_resumen = pd.DataFrame(data_dict)
        dfs_resumen.append(df_resumen)
    # Inicia script para guardar en excel
    writer = pd.ExcelWriter(os.path.join(outputdir, outputfile), engine='xlsxwriter')
    for df, sheet in zip(dfs_resumen, ys):
        df.to_excel(writer, sheet_name = sheet, index = False)
    writer.save()
    # Finaliza script para guardar en excel
    return dfs_resumen

## Homogeneidad ELO

In [5]:
datadir = os.path.join(os.path.pardir,'datos','regresiones')
ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
outputdir = os.path.join(os.path.pardir,'resultados','homogeneidad-categorias')
outputfile = 'homogeneidad-elo.xlsx'
dfs_resumen = anova_1via_categorias(datadir,
                                    ligas,
                                    outputdir,
                                    outputfile,
                                    column_condition = lambda x: 'categoria' in x and 'elo' in x)

## Homogeneidad General

In [6]:
datadir = os.path.join(os.path.pardir,'datos','regresiones')
ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
outputdir = os.path.join(os.path.pardir,'resultados','homogeneidad-categorias')
outputfile = 'homogeneidad-general.xlsx'
dfs_resumen = anova_1via_categorias(datadir,
                                    ligas,
                                    outputdir,
                                    outputfile,
                                    column_condition = lambda x: 'categoria' in x and 'general' in x)

## Homogeneidad Local

In [7]:
datadir = os.path.join(os.path.pardir,'datos','regresiones')
ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
outputdir = os.path.join(os.path.pardir,'resultados','homogeneidad-categorias')
outputfile = 'homogeneidad-local.xlsx'
dfs_resumen = anova_1via_categorias(datadir,
                                    ligas,
                                    outputdir,
                                    outputfile,
                                    column_condition = lambda x: 'categoria' in x and 'local' in x)

# Homogeneidad Visita

In [8]:
datadir = os.path.join(os.path.pardir,'datos','regresiones')
ligas = ['Alemania','Espana','Francia','Inglaterra','Italia']
outputdir = os.path.join(os.path.pardir,'resultados','homogeneidad-categorias')
outputfile = 'homogeneidad-visita.xlsx'
dfs_resumen = anova_1via_categorias(datadir,
                                    ligas,
                                    outputdir,
                                    outputfile,
                                    column_condition = lambda x: 'categoria' in x and 'visita' in x)